In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt
import pickle

In [3]:
MNM_nb_folder = os.path.join('..', '..', '..', 'side_project', 'network_builder')
sys.path.append(MNM_nb_folder)
python_lib_folder = os.path.join('..', '..', 'pylib')
sys.path.append(python_lib_folder)

In [4]:
from MNM_nb import *
import MNMAPI
from sDODE import *

In [5]:
data_folder = os.path.join('..', '..', '..', 'data', 'input_files_SR41_fix2')

In [6]:
nb = MNM_network_builder()
nb.load_from_folder(data_folder)
usefuk_link_list = list(filter(lambda x: x.typ == 'CTM', nb.link_list))

MNM_config
MNM_pathtable


In [7]:
nb.od

MNM_od, number of O:174, number of D:174

In [8]:
len(usefuk_link_list)

1717

In [9]:
num_observed_links = 500

In [10]:
config = dict()

config['use_link_flow'] = True
config['use_link_tt'] = False
config['link_flow_weight'] = 1
config['link_tt_weight'] = 1
config['num_data'] = 10
config['observed_links'] = list(map(lambda x: x.ID, np.random.choice(usefuk_link_list, num_observed_links)))
config['paths_list'] = list(range(nb.config.config_dict['FIXED']['num_path']))

In [11]:
# pickle.dump([true_q_para, sdode], open('sr41logit_true2.pickle', 'wb'))

In [12]:
[true_q_para, sdode] = pickle.load(open('sr41logit_true2.pickle', 'rb'))
config = sdode.config

In [13]:
num_interval = nb.config.config_dict['DTA']['max_interval']
theta = 0.01

In [14]:
O_dist = np.arange(174)
D_dist = np.arange(174)

In [15]:
sdode = SDODE(nb, config)

In [16]:
demand_mean_list  = list()
for i in range(sdode.num_assign_interval):
    tmp_arr = np.zeros(len(nb.demand.demand_list))
    for j, (O,D) in enumerate(nb.demand.demand_list):
        tmp_arr[j] = nb.demand.demand_dict[O][D][i]
    demand_mean_list.append(tmp_arr)

In [17]:
# true_q = sdode.init_demand_flow(init_scale = 50)
true_q_para = OD_parameter_server(sdode.demand_list, sdode.num_assign_interval)
true_q_para.construct(O_dist, D_dist)
true_q_para.initialize(mean_scale = 5, std_scale= 1, 
                        O_cov_scale = 5, D_cov_scale = 5)
true_q_para.demand_mean_list = demand_mean_list
# true_q_para = pickle.load(open('potential_true_q_312.pickle', 'rb'))
true_q = np.concatenate(true_q_para.demand_mean_list)
old_f = np.zeros(num_interval * len(config['paths_list']))
num_iters = 50
P = sdode.nb.get_route_portion_matrix()
true_f = P.dot(true_q)
for i in range(num_iters):
    print (i)
    dta = sdode._run_simulation(true_f)
    path_cost = dta.get_path_tt(np.arange(0, sdode.num_loading_interval, sdode.ass_freq))
    sdode.assign_route_portions(path_cost, theta = theta)
    new_f = sdode.nb.get_route_portion_matrix().dot(true_q)
    true_f = np.float(i) / np.float(i+1) * true_f + np.float(1) / np.float(i+1) * new_f
    print np.linalg.norm(true_f - old_f)
    old_f = true_f
    sdode.nb.update_demand_path(true_f)

0
2111.8028706276714
1
29.011742010099645
2
10.936786318675548
3
7.180889189367931
4
6.81193878600006
5
3.996168708849766
6
4.223958423127326
7
3.013801276950867
8
2.6805867061389064
9
2.5227079073165934
10
2.1637480230337127
11
1.814756052264904
12
1.4789576364999795
13
1.5478726036464918
14
1.7822000623543466
15
1.8377797635819006
16
1.5759568541517446
17
1.433113519927691
18
1.1919754385347956
19
0.9170455545791618
20
1.0028108088858951
21
0.9679012621200532
22
0.9585173032420932
23
0.8833708429997181
24
0.7913975835174335
25
0.8069547761033353
26
0.8786485662618839
27
0.7246472171204801
28
0.6710752138244855
29
0.7135753482302164
30
0.7844428474815694
31
0.7701181381072896
32
0.6134594215427318
33
0.5636078134021463
34
0.5077478264806247
35
0.6431608434547996
36
0.5236642658283416
37
0.5836129401352741
38
0.5107976006590705
39
0.7766074296472658
40
0.5768933894675221
41
0.4640700787833636
42
0.5044873982557792
43
0.42348706249517376
44
0.4380106677930045
45
0.4204030921530316
46
0.

In [17]:
true_q = np.concatenate(true_q_para.demand_mean_list)
P = sdode.nb.get_route_portion_matrix()
true_f = P.dot(true_q)

In [18]:
assert (np.isclose(true_q.sum(), true_f.sum()))

In [19]:
dta = sdode._run_simulation(true_f)
true_P = sdode.nb.get_route_portion_matrix()
true_full_dar = sdode.get_full_dar(dta, true_f)
true_dar = true_full_dar[sdode.get_full_observed_link_index(),:]
full_true_x = true_full_dar.dot(true_f)
true_x = true_dar.dot(true_f)
true_path_cost = dta.get_path_tt(np.arange(0, sdode.num_loading_interval, sdode.ass_freq))

In [24]:
num_data = config['num_data']
data_dict = dict()
data_dict['link_flow'] = list()
sdode = SDODE(nb, config)
for i in range(config['num_data']):
    print i
    q_e, _ = true_q_para.forward()
    f_e = true_P.dot(q_e)
    dta = sdode._run_simulation(f_e)
    full_dar_e = sdode.get_full_dar(dta, f_e)
    dar_e = full_dar_e[sdode.get_full_observed_link_index(),:]
    x_e = dar_e.dot(f_e)
    link_df = pd.DataFrame(index = range(num_interval), columns = config['observed_links'], 
                           data = x_e.reshape((num_interval, len(config['observed_links']))) 
                               + np.random.randn(num_interval, len(config['observed_links'])) * np.sqrt(5))
    data_dict['link_flow'].append(link_df)

0
1
2
3
4
5
6
7
8
9


In [18]:
# pickle.dump(data_dict, open('sr41data_dict2.pickle', 'wb'))

In [20]:
data_dict = pickle.load(open('sr41data_dict2.pickle', 'rb'))

In [21]:
data_dict['link_flow'][0]

,3166,3039,2064,1829,433,1997,3314,387,2559,225,...,210,1934,455,2165,621,1653,1851,306,2376,2516
0,0.933215,44.096235,6.617581,26.804392,300.590947,13.444423,33.910333,-2.376275,29.026397,1.059567,...,51.565593,73.279561,47.967256,5.155859,149.271291,58.121083,-0.614852,4.235266,28.197840,5.762159
1,0.027531,131.622622,13.801112,72.289504,877.828203,16.692283,107.625842,2.066398,62.117405,-1.943102,...,145.764928,194.750938,122.603658,9.030619,283.974403,149.953579,-2.765423,-4.420281,72.871183,15.216627
2,0.839189,215.738643,27.373931,115.537538,1169.257649,25.902923,173.320254,8.049181,108.165936,2.990379,...,183.627486,328.035450,96.078989,12.050779,440.336725,228.428989,2.844831,6.825674,109.056514,23.465949
3,1.547208,224.421836,24.994988,122.897750,1414.936539,25.515151,188.176786,6.413582,104.770771,0.038963,...,148.701557,318.081974,215.745677,11.834827,452.499899,234.435159,6.364276,-1.272605,115.105762,24.889991
4,2.011152,148.618954,14.376901,81.971594,1282.895257,22.023680,110.248597,3.737054,64.835295,-1.010721,...,366.140989,198.935088,354.545411,11.508502,469.760388,154.166178,-1.955835,-2.590409,80.147994,19.520458
5,0.701286,73.639518,2.323036,47.401660,321.502416,8.349586,42.479173,2.459776,26.438330,-3.003869,...,74.721998,80.271936,108.541034,7.693261,374.226551,58.994355,3.424223,1.801457,32.030004,7.669435


In [22]:
3166 in config['observed_links']

True

In [23]:
sdode = SDODE(nb, config)

In [24]:
sdode.add_data(data_dict)

In [ ]:
print "Start sovling"
q_para, loss_list1 = sdode.estimate_demand_cov(O_dist, D_dist, init_mean_scale = 1, 
                      init_std_scale = 0.1, init_O_cov_scale = 0.1,
                      init_D_cov_scale = 0.1, step_size = 1, theta = theta, max_epoch = 200,
                                  adagrad = True, known_path_cost = None, save_folder = 'sr41save')

Start sovling
Epoch: 0 Loss: 5710.3853656789
Epoch: 1 Loss: 4214.60942913494
Epoch: 2 Loss: 3675.7060493627823
Epoch: 3 Loss: 3311.223780584838
Epoch: 4 Loss: 3085.932408706243
Epoch: 5 Loss: 2890.609780837973
Epoch: 6 Loss: 2751.31110538753
Epoch: 7 Loss: 2592.356344915329
Epoch: 8 Loss: 2514.423570309513
Epoch: 9 Loss: 2396.627682531728
Epoch: 10 Loss: 2433.960032678087
Epoch: 11 Loss: 2339.703728091794
Epoch: 12 Loss: 2332.348880899595
Epoch: 13 Loss: 2240.144824143214
Epoch: 14 Loss: 2189.1795602360808
Epoch: 15 Loss: 2185.1376911822276
Epoch: 16 Loss: 2166.021303244855
Epoch: 17 Loss: 2133.783644464664
Epoch: 18 Loss: 2176.919441876263
Epoch: 19 Loss: 2204.9298088835026
Epoch: 20 Loss: 2113.825419983451
Epoch: 21 Loss: 2122.4514287389875
Epoch: 22 Loss: 2096.808699757723
Epoch: 23 Loss: 2102.8148135173287
Epoch: 24 Loss: 2073.621973603769
Epoch: 25 Loss: 2082.9297106339445
Epoch: 26 Loss: 2039.1627452285472
Epoch: 27 Loss: 2062.329025491345
Epoch: 28 Loss: 2095.071451178016
Epoch: